## import dependencies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

# Load Cats vs Dogs dataset with train, validation, test splits

In [2]:
(train, validation, test), metadata = tfds.load(
                                        'cats_vs_dogs',
                                        split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
                                        with_info=True,
                                        as_supervised=True
                                      )
num_examples = metadata.splits['train'].num_examples
num_classes = metadata.features['label'].num_classes
print(num_examples)
print(num_classes)

23262
2


# Resize and normalize images

In [3]:
IMAGE_SIZE = (224, 224)

def format_image(image, label):
    image = tf.image.resize(image, IMAGE_SIZE) / 255.0
    return image, label

# Prepare batches for training, validation, and testing

In [4]:
BATCH_SIZE = 32

## Training Batch
train_batches = train.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)

## Validation Batch
validation_batches = validation.map(format_image).batch(BATCH_SIZE).prefetch(1)

## Test batch
test_batches = test.map(format_image).batch(1)

# Load MobileNetV2 base model without top layers, pretrained on ImageNet

In [5]:
base_model = tf.keras.applications.MobileNetV2(
        input_shape=IMAGE_SIZE + (3,),
        include_top=False,
        weights='imagenet',
        pooling='avg'  # Global average pooling
    )

    # Freeze the base model
base_model.trainable = False

    # Build the model
model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

# Compile and train

In [6]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMAGE_SIZE + (3,),
                                               include_top=False,
                                               weights='imagenet')

# Freeze the base model
base_model.trainable = False

# Build the model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_batches, epochs=5, validation_data=validation_batches)

Epoch 1/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 54s 67ms/step - accuracy: 0.9536 - loss: 0.1205 - val_accuracy: 0.9815 - val_loss: 0.0474
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 72s 55ms/step - accuracy: 0.9854 - loss: 0.0420 - val_accuracy: 0.9824 - val_loss: 0.0466
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 52s 79ms/step - accuracy: 0.9867 - loss: 0.0368 - val_accuracy: 0.9824 - val_loss: 0.0524
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 47s 65ms/step - accuracy: 0.9873 - loss: 0.0338 - val_accuracy: 0.9815 - val_loss: 0.0561
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 95s 85ms/step - accuracy: 0.9895 - loss: 0.0299 - val_accuracy: 0.9802 - val_loss: 0.0568


# Save model in SavedModel format

In [7]:
PetVision_model = "/content/drive/MyDrive/PetVision-TFLite/pet_vision_model"
tf.saved_model.save(model, PetVision_model)

# Convert to TensorFlow Lite model

In [9]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

tflite_model_file = '/content/drive/MyDrive/PetVision-TFLite/pet_vision_model.tflite'
with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmpb8cepii3'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_312')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  134937415815376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134937406991120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134937406990160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134937406990928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134937406990352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134937406989776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134937406986896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134937406988816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134937406990544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134937406992080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1349374069

# Load TFLite interpreter and allocate tensors

In [10]:
interpreter = tf.lite.Interpreter(model_path = tflite_model_file)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

predictions = []

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


# Run inference on first 100 test images using TFLite interpreter

In [11]:
test_labels, test_imgs = [], []
for img, label in test_batches.take(100):
  interpreter.set_tensor(input_index, img)
  interpreter.invoke()
  predictions.append(interpreter.get_tensor(output_index))
  test_labels.append(label.numpy()[0])
  test_imgs.append(img)

# Calculate number of correct predictions out of 100

In [12]:
score = 0
for item in range(0,99):
  prediction=np.argmax(predictions[item])
  label = test_labels[item]
  if prediction==label:
    score=score+1
print("Out of 100 predictions I got " + str(score) + " correct")

Out of 100 predictions I got 99 correct


# Define function to plot a single image with prediction info

In [13]:
def plot_image(i, predictions, true_labels, images):
    predictions_array = np.array(predictions[i][0])
    true_label = true_labels[i]
    img = images[i][0]

    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel(
        "Predicted: {} ({:2.0f}%)\nTrue: {}".format(
            predicted_label,
            100 * np.max(predictions_array),
            true_label
        ),
        color=color
    )


# Plot test images in a grid, 3 images per row, using existing plot_image function

In [14]:
num_images = 30
images_per_row = 3
rows = num_images // images_per_row + int(num_images % images_per_row > 0)

plt.figure(figsize=(5 * images_per_row, 4 * rows))

for i in range(num_images):
    plt.subplot(rows, images_per_row, i + 1)
    plot_image(i, predictions, test_labels, test_imgs)

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

## Optimize the model

In [21]:
## dynamic range quantization

converter  = tf.lite.TFLiteConverter.from_saved_model(PetVision_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model_dynamic = converter.convert()
tflite_model_dynamic_file = '/content/drive/MyDrive/PetVision-TFLite/dynamic_optimized_model.tflite'

with open(tflite_model_dynamic_file, 'wb') as file:
  file.write(tflite_model_dynamic)

In [23]:
## integer quantization

converter  = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

def representative_data_gen():
    for input_value, _ in test_batches.take(100):
        yield [tf.cast(input_value, tf.float32)]

converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model_integer = converter.convert()
tflite_model_integer_file = '/content/drive/MyDrive/PetVision-TFLite/integer_optimized_model.tflite'

with open(tflite_model_integer_file, 'wb') as file:
  file.write(tflite_model_integer)

Saved artifact at '/tmp/tmpswpz87e3'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_312')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  134937415815376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134937406991120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134937406990160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134937406990928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134937406990352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134937406989776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134937406986896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134937406988816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134937406990544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134937406992080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1349374069

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
